In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns

from matplotlib import pyplot as plt
from medcat.cat import CAT

C:\Users\BILAL\AppData\Roaming\Python\Python37\site-packages\medcat\cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Load model pack and Create CAT - the main class from medcat used for concept annotation

model_pack_path = 'medcat/mc_modelpack_snomed_int_16_mar_2022_25be3857ba34bdd5'
cat = CAT.load_model_pack(model_pack_path)

Found an existing unziped model pack at: medcat\mc_modelpack_snomed_int_16_mar_2022_25be3857ba34bdd5, the provided zip will not be touched.
{
  "Model ID": "25be3857ba34bdd5",
  "Last Modifed On": "16 March 2022",
  "History (from least to most recent)": [
    "a474096eb4566638",
    "009617d7ff372682"
  ],
  "Description": "SNOMED INT enriched with UMLS and trained unsupervised on MIMIC-III",
  "Source Ontology": "SnomedCT_InternationalRF2_PRODUCTION_20210131T120000Z",
  "Location": "MedCAT Rosalind machine, available for public download from https://github.com/CogStack/MedCAT",
  "MetaCAT models": {
    "Status": "Detects is a concept Affirmed or Negated/Hypothetical"
  },
  "Basic CDB Stats": {
    "Number of concepts": 354448,
    "Number of names": 2049216,
    "Number of concepts that received training": 29674,
    "Number of seen training examples in total": 20585988,
    "Average training examples per concept": 693.7382220125362
  },
  "Performance": {
    "ner": {},
    "meta"

In [3]:
test_notes = pd.read_csv('MIMIC_IV_clean_test_set_4digitcode.csv')
test_notes

,Unnamed: 0,hadm_id,text,subject_id,seq_num,icd_code,icd_version
0,0,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,1.0,D50.0,10.0
1,1,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,2.0,K52.1,10.0
2,2,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,3.0,I10.,10.0
3,3,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,4.0,E53.8,10.0
4,4,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,5.0,M81.0,10.0
...,...,...,...,...,...,...,...
259313,259313,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,13.0,D72.8,10.0
259314,259314,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,14.0,I25.2,10.0
259315,259315,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,15.0,G25.8,10.0
259316,259316,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,16.0,M19.9,10.0


In [8]:
test_notes.drop(columns = ['Unnamed: 0'], inplace = True)

KeyError: "['Unnamed: 0'] not found in axis"

In [9]:
unique_hadm_notes = test_notes[['hadm_id', 'text']].drop_duplicates(subset='hadm_id')


In [10]:
unique_hadm_notes

,hadm_id,text
0,20000024,sex f medicine allergies aspirin chief complai...
10,20002636,sex m medicine allergies prevacid ciprofloxaci...
27,20002800,sex f medicine allergies penicillins tetanus t...
58,20003013,sex m neurology allergies no known allergies a...
71,20003425,sex m otolaryngology allergies no known allerg...
...,...,...
259249,29996653,sex f medicine allergies erythromycin base ato...
259281,29996750,sex f medicine allergies ciprofloxacin codeine...
259289,29997523,sex f surgery allergies no known allergies adv...
259295,29997858,sex m medicine allergies no known allergies ad...


In [11]:
unique_hadm_notes = unique_hadm_notes.reset_index(drop = True)

In [12]:
icd10_data = {
    'hadm_id': [],
    'icd10_codes': []
}



In [13]:
i = 0
for index, row in unique_hadm_notes.iterrows():
    i +=1
    
    if i%100 ==0:
        print(f'{i} notes have been processed!', '\n')
        
        
    single_note = row['text']
    hadm_id = row['hadm_id']

    # Get the entities from the text
    all_entities = cat.get_entities(single_note)

    icd_codes = set()  # Using a set to automatically remove duplicates

    for entity_id, entity_data in all_entities['entities'].items():
        icd10 = entity_data.get('icd10', [])
        for code in icd10:
            if code:  # Check if the code is not an empty string
                icd_codes.add(code)

    icd10_data['hadm_id'].append(hadm_id)
    icd10_data['icd10_codes'].append(list(icd_codes))



100 notes have been processed! 

200 notes have been processed! 

300 notes have been processed! 

400 notes have been processed! 

500 notes have been processed! 

600 notes have been processed! 

700 notes have been processed! 

800 notes have been processed! 

900 notes have been processed! 

1000 notes have been processed! 

1100 notes have been processed! 

1200 notes have been processed! 

1300 notes have been processed! 

1400 notes have been processed! 

1500 notes have been processed! 

1600 notes have been processed! 

1700 notes have been processed! 

1800 notes have been processed! 

1900 notes have been processed! 

2000 notes have been processed! 

2100 notes have been processed! 

2200 notes have been processed! 

2300 notes have been processed! 

2400 notes have been processed! 

2500 notes have been processed! 

2600 notes have been processed! 

2700 notes have been processed! 

2800 notes have been processed! 

2900 notes have been processed! 

3000 notes have been pr

In [14]:
# Create a DataFrame from the processed data
df_icd10 = pd.DataFrame(icd10_data)
df_icd10

,hadm_id,icd10_codes
0,20000024,"[M81.99, K66.1, R20.8, K52.9, K44.9, J30.4, N8..."
1,20002636,"[I51.8, R57.9, I42.9, I31.3, M10.99, G47.3, E7..."
2,20002800,"[I51.8, R35, F32.2, L03.9, N83.2, G62.9, R09.0..."
3,20003013,"[I64, R41.3, R50.8, R05, R25.2, A09.9, R47.0, ..."
4,20003425,"[T14.0, T81.9, R52.9, Z48.9, R06.2, R22.1, R29..."
...,...,...
18340,29996653,"[R13, J81, K11.7, I64, R05, G62.9, E78.5, E04...."
18341,29996750,"[R94.3, E27.8, I64, K44.9, E78.8, R50.8, Z73.3..."
18342,29997523,"[R33, I50.0, R63.0, R58, D64.9, R22.2, Z86.2, ..."
18343,29997858,"[R63.4, K57.9, R05, M25.59, R58, I51.9, R21, Z..."


In [15]:
# Define the reformat function
def reformat(code):
    code = ''.join(code.split('.'))
    if len(code) >3:
        code = code[:3] + '.' + code[3]
    else:
        code = code[:3] + '.'
    return code




In [16]:

# Apply the reformat function to each element in the 'icd10_codes' column
df_icd10['icd10_codes'] = df_icd10['icd10_codes'].apply(lambda codes: [reformat(code) for code in codes])
df_icd10

,hadm_id,icd10_codes
0,20000024,"[M81.9, K66.1, R20.8, K52.9, K44.9, J30.4, N83..."
1,20002636,"[I51.8, R57.9, I42.9, I31.3, M10.9, G47.3, E78..."
2,20002800,"[I51.8, R35., F32.2, L03.9, N83.2, G62.9, R09...."
3,20003013,"[I64., R41.3, R50.8, R05., R25.2, A09.9, R47.0..."
4,20003425,"[T14.0, T81.9, R52.9, Z48.9, R06.2, R22.1, R29..."
...,...,...
18340,29996653,"[R13., J81., K11.7, I64., R05., G62.9, E78.5, ..."
18341,29996750,"[R94.3, E27.8, I64., K44.9, E78.8, R50.8, Z73...."
18342,29997523,"[R33., I50.0, R63.0, R58., D64.9, R22.2, Z86.2..."
18343,29997858,"[R63.4, K57.9, R05., M25.5, R58., I51.9, R21.,..."


In [17]:
# Define a function to remove duplicates within a list
def remove_duplicates(codes):
    seen = set()
    result = []
    for code in codes:
        if code not in seen:
            result.append(code)
            seen.add(code)
    return result

In [18]:

# Apply the remove_duplicates function to the 'icd10_codes' column for each 'hadm_id'
df_icd10['icd10_codes'] = df_icd10['icd10_codes'].apply(remove_duplicates)
df_icd10

,hadm_id,icd10_codes
0,20000024,"[M81.9, K66.1, R20.8, K52.9, K44.9, J30.4, N83..."
1,20002636,"[I51.8, R57.9, I42.9, I31.3, M10.9, G47.3, E78..."
2,20002800,"[I51.8, R35., F32.2, L03.9, N83.2, G62.9, R09...."
3,20003013,"[I64., R41.3, R50.8, R05., R25.2, A09.9, R47.0..."
4,20003425,"[T14.0, T81.9, R52.9, Z48.9, R06.2, R22.1, R29..."
...,...,...
18340,29996653,"[R13., J81., K11.7, I64., R05., G62.9, E78.5, ..."
18341,29996750,"[R94.3, E27.8, I64., K44.9, E78.8, R50.8, Z73...."
18342,29997523,"[R33., I50.0, R63.0, R58., D64.9, R22.2, Z86.2..."
18343,29997858,"[R63.4, K57.9, R05., M25.5, R58., I51.9, R21.,..."


In [19]:
df_icd10.to_csv('test_set_mapped_icd_hadmids_18k_4digits.csv')

unique_hadm_notes.to_csv('test_set_unique_notes_hadmids_18k_4digits.csv')


In [20]:
df_icd10.dtypes

hadm_id         int64
icd10_codes    object
dtype: object

In [21]:
np.array(df_icd10[df_icd10.hadm_id==20000024].icd10_codes[0])

array(['M81.9', 'K66.1', 'R20.8', 'K52.9', 'K44.9', 'J30.4', 'N83.8',
       'K57.9', 'K58.9', 'H40.9', 'A09.9', 'R58.', 'D64.9', 'K40.9',
       'Z88.9', 'Q89.9', 'R23.0', 'R01.2', 'R60.9', 'C22.9', 'C34.9',
       'R19.8', 'T14.0', 'C50.9', 'R32.', 'R01.1', 'R52.9', 'Z82.4',
       'H35.3', 'R29.6', 'R27.0', 'S32.1', 'R71.', 'R06.2', 'R09.8',
       'K59.0', 'M19.9', 'R53.'], dtype='<U5')

In [22]:
test_notes[test_notes.hadm_id == 20000024].icd_code.unique().astype('str')

array(['D50.0', 'K52.1', 'I10.', 'E53.8', 'M81.0', 'R27.0', 'Z91.8',
       'H54.8', 'T47.4', 'Y92.0'], dtype='<U5')

In [23]:
df_icd10.drop_duplicates(subset = ['hadm_id'], inplace = True)

In [24]:
test_notes

,hadm_id,text,subject_id,seq_num,icd_code,icd_version
0,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,1.0,D50.0,10.0
1,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,2.0,K52.1,10.0
2,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,3.0,I10.,10.0
3,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,4.0,E53.8,10.0
4,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,5.0,M81.0,10.0
...,...,...,...,...,...,...
259313,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,13.0,D72.8,10.0
259314,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,14.0,I25.2,10.0
259315,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,15.0,G25.8,10.0
259316,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,16.0,M19.9,10.0


In [25]:

unique_icd_codes = set(test_notes['icd_code']) #.union(set(icd_code for icd_list in df_icd10['icd10_codes'] for icd_code in icd_list))



In [26]:
len(unique_icd_codes)

4330

In [27]:
test_notes.icd_code.nunique()

4330

In [28]:
# Step 3: Initialize TP, FP, FN, TN counts
counts = {icd_code: (0, 0, 0, 0) for icd_code in unique_icd_codes}



In [29]:
i = 0
for icd_code in unique_icd_codes:
    i+=1
    if i%100 == 0:
        print(f'{i} codes are done!')
    
    true_icd_codes = test_notes[test_notes['icd_code'] == icd_code]['hadm_id'].values
    predicted_icd_codes = df_icd10[df_icd10['icd10_codes'].apply(lambda x: icd_code in x)]['hadm_id'].values

    TP = np.intersect1d(true_icd_codes, predicted_icd_codes)
    FP = np.setdiff1d(predicted_icd_codes, true_icd_codes)
    FN = np.setdiff1d(true_icd_codes, predicted_icd_codes)
    TN = len(np.union1d(predicted_icd_codes, true_icd_codes)) -len(TP) -len(FP) -len(FN)


    counts[icd_code] = (len(TP), len(FP), len(FN), TN)

100 codes are done!
200 codes are done!
300 codes are done!
400 codes are done!
500 codes are done!
600 codes are done!
700 codes are done!
800 codes are done!
900 codes are done!
1000 codes are done!
1100 codes are done!
1200 codes are done!
1300 codes are done!
1400 codes are done!
1500 codes are done!
1600 codes are done!
1700 codes are done!
1800 codes are done!
1900 codes are done!
2000 codes are done!
2100 codes are done!
2200 codes are done!
2300 codes are done!
2400 codes are done!
2500 codes are done!
2600 codes are done!
2700 codes are done!
2800 codes are done!
2900 codes are done!
3000 codes are done!
3100 codes are done!
3200 codes are done!
3300 codes are done!
3400 codes are done!
3500 codes are done!
3600 codes are done!
3700 codes are done!
3800 codes are done!
3900 codes are done!
4000 codes are done!
4100 codes are done!
4200 codes are done!
4300 codes are done!


In [30]:
counts

{'T86.2': (0, 0, 3, 0),
 'C71.9': (2, 18, 10, 0),
 'I47.1': (159, 280, 146, 0),
 'D25.1': (1, 0, 28, 0),
 'Z53.8': (0, 0, 22, 0),
 'F32.2': (7, 286, 3, 0),
 'I73.8': (0, 76, 7, 0),
 'L25.8': (0, 0, 3, 0),
 'M46.1': (7, 17, 2, 0),
 'Q63.8': (0, 1, 1, 0),
 'R23.4': (3, 756, 8, 0),
 'F31.5': (0, 0, 9, 0),
 'D12.0': (0, 0, 22, 0),
 'N50.8': (15, 104, 12, 0),
 'R47.0': (176, 1146, 31, 0),
 'G37.9': (1, 21, 2, 0),
 'Z91.0': (1, 236, 37, 0),
 'K25.4': (5, 13, 43, 0),
 'C48.1': (0, 0, 5, 0),
 'C32.0': (1, 2, 2, 0),
 'N72.': (0, 0, 1, 0),
 'C91.3': (0, 0, 2, 0),
 'G40.5': (0, 8, 5, 0),
 'L02.5': (0, 0, 16, 0),
 'C09.9': (1, 4, 1, 0),
 'G30.9': (136, 204, 27, 0),
 'N28.9': (4, 1018, 44, 0),
 'G97.3': (0, 0, 3, 0),
 'N13.0': (0, 1, 10, 0),
 'W10.1': (0, 0, 11, 0),
 'Q65.8': (4, 3, 2, 0),
 'O24.1': (0, 0, 6, 0),
 'M75.0': (2, 31, 8, 0),
 'I15.1': (0, 1, 4, 0),
 'R25.9': (0, 0, 3, 0),
 'D47.1': (4, 42, 2, 0),
 'Y83.1': (0, 1, 225, 0),
 'F40.0': (20, 16, 1, 0),
 'R35.8': (0, 0, 11, 0),
 'O36.8': (5,

In [31]:



# Calculate Precision and Recall for Each ICD-10 Code
precisions = {}
recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    precisions[icd_code] = precision
    recalls[icd_code] = recall


In [32]:
precisions

{'T86.2': 0,
 'C71.9': 0.1,
 'I47.1': 0.3621867881548975,
 'D25.1': 1.0,
 'Z53.8': 0,
 'F32.2': 0.023890784982935155,
 'I73.8': 0.0,
 'L25.8': 0,
 'M46.1': 0.2916666666666667,
 'Q63.8': 0.0,
 'R23.4': 0.003952569169960474,
 'F31.5': 0,
 'D12.0': 0,
 'N50.8': 0.12605042016806722,
 'R47.0': 0.13313161875945537,
 'G37.9': 0.045454545454545456,
 'Z91.0': 0.004219409282700422,
 'K25.4': 0.2777777777777778,
 'C48.1': 0,
 'C32.0': 0.3333333333333333,
 'N72.': 0,
 'C91.3': 0,
 'G40.5': 0.0,
 'L02.5': 0,
 'C09.9': 0.2,
 'G30.9': 0.4,
 'N28.9': 0.003913894324853229,
 'G97.3': 0,
 'N13.0': 0.0,
 'W10.1': 0,
 'Q65.8': 0.5714285714285714,
 'O24.1': 0,
 'M75.0': 0.06060606060606061,
 'I15.1': 0.0,
 'R25.9': 0,
 'D47.1': 0.08695652173913043,
 'Y83.1': 0.0,
 'F40.0': 0.5555555555555556,
 'R35.8': 0,
 'O36.8': 0.08928571428571429,
 'G44.3': 0.5,
 'V23.4': 0,
 'L65.9': 0.022727272727272728,
 'L89.6': 0,
 'R93.4': 0,
 'R40.3': 0,
 'R03.0': 0.06666666666666667,
 'H53.4': 0.3237410071942446,
 'M17.5': 0,
 

In [33]:
# Step 6: Calculate Macro Precision and Macro Recall
macro_precision = sum(precisions.values()) / len(precisions)
macro_recall = sum(recalls.values()) / len(recalls)


In [34]:
macro_precision

0.13584406189891207

In [35]:
macro_recall

0.2298904471107249

In [36]:
# Calculate F1 Score for Each ICD-10 Code
f1_scores = {}
for icd_code in unique_icd_codes:
    precision = precisions[icd_code]
    recall = recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    f1_scores[icd_code] = f1_score



In [37]:
# Step 8: Calculate Macro F1 Score
macro_f1_score = sum(f1_scores.values()) / len(f1_scores)

# Now you have macro precision, macro recall, and macro F1 score for your multi-label classification evaluation.

In [38]:
macro_f1_score

0.1279583553797011

In [39]:
# Calculate Precision and Recall for Each ICD-10 Code
precisions = {}
recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    precisions[icd_code] = precision
    recalls[icd_code] = recall

# Filter out zero precision values and calculate average precision
average_precision = np.mean([precision for precision in precisions.values() if precision > 0])

In [40]:
average_precision

0.3152222872573895

In [41]:
average_recall = np.mean([recall for recall in recalls.values() if recall > 0])

In [42]:
average_recall

0.5334542529418212

In [43]:
# Calculate F1 Score for Each ICD-10 Code
f1_scores = {}
for icd_code in unique_icd_codes:
    precision = precisions[icd_code]
    recall = recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    f1_scores[icd_code] = f1_score


In [44]:
average_f1 = np.mean([f1_score for f1_score in f1_scores.values() if f1_score > 0])

In [45]:
average_f1

0.2969237292572908

In [46]:
len([f1_score for f1_score in f1_scores.values() if f1_score > 0])

1866

#### Top 50 ICD codes identification

In [47]:
top_50_icd_codes = test_notes['icd_code'].value_counts().index[:50]
top_50_icd_codes

Index(['E78.5', 'I10.', 'Z87.8', 'K21.9', 'I25.1', 'Z79.0', 'F32.9', 'F41.9',
       'N17.9', 'Z86.7', 'Z79.4', 'E03.9', 'Y92.2', 'E11.9', 'G47.3', 'F17.2',
       'I48.9', 'D64.9', 'E66.9', 'J45.9', 'I50.3', 'Y92.9', 'Z66.', 'N39.0',
       'E11.2', 'D62.', 'J44.9', 'I12.9', 'K59.0', 'Z68.3', 'I50.2', 'N18.9',
       'E87.1', 'I25.2', 'Y92.0', 'Z95.5', 'G89.2', 'E87.2', 'E11.6', 'I11.0',
       'J96.0', 'Z85.8', 'N40.0', 'N18.3', 'Z96.6', 'E11.4', 'I13.0', 'D72.8',
       'I48.0', 'G47.0'],
      dtype='object')

In [48]:
filtered_test_notes = test_notes[test_notes['icd_code'].isin(top_50_icd_codes)]

filtered_test_notes

,hadm_id,text,subject_id,seq_num,icd_code,icd_version
2,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,3.0,I10.,10.0
9,20000024,sex f medicine allergies aspirin chief complai...,16925328.0,10.0,Y92.0,10.0
11,20002636,sex m medicine allergies prevacid ciprofloxaci...,12527107.0,2.0,N17.9,10.0
13,20002636,sex m medicine allergies prevacid ciprofloxaci...,12527107.0,4.0,I50.2,10.0
14,20002636,sex m medicine allergies prevacid ciprofloxaci...,12527107.0,5.0,Z85.8,10.0
...,...,...,...,...,...,...
259308,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,8.0,I25.1,10.0
259311,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,11.0,E03.9,10.0
259312,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,12.0,K21.9,10.0
259313,29998079,sex f medicine allergies sulfamethoxazole prav...,16957952.0,13.0,D72.8,10.0


In [49]:
filtered_unique_hadm_notes = filtered_test_notes[['hadm_id', 'text']].drop_duplicates(subset='hadm_id')


In [50]:
filtered_unique_hadm_notes = filtered_unique_hadm_notes.reset_index(drop = True)
filtered_unique_hadm_notes

,hadm_id,text
0,20000024,sex f medicine allergies aspirin chief complai...
1,20002636,sex m medicine allergies prevacid ciprofloxaci...
2,20002800,sex f medicine allergies penicillins tetanus t...
3,20003013,sex m neurology allergies no known allergies a...
4,20003425,sex m otolaryngology allergies no known allerg...
...,...,...
17353,29996653,sex f medicine allergies erythromycin base ato...
17354,29996750,sex f medicine allergies ciprofloxacin codeine...
17355,29997523,sex f surgery allergies no known allergies adv...
17356,29997858,sex m medicine allergies no known allergies ad...


In [51]:
filtered_icd10_data = {
    'hadm_id': [],
    'icd10_codes': []
}



In [52]:
unique_icd_codes = set(filtered_test_notes['icd_code'])

In [53]:
# Step 3: Initialize TP, FP, FN, TN counts
filtered_counts = {icd_code: (0, 0, 0, 0) for icd_code in unique_icd_codes}



In [54]:
i = 0
for icd_code in unique_icd_codes:
    i+=1
    if i%10 == 0:
        print(f'{i} codes are done!')
    
    true_icd_codes = filtered_test_notes[filtered_test_notes['icd_code'] == icd_code]['hadm_id'].values
    predicted_icd_codes = df_icd10[df_icd10['icd10_codes'].apply(lambda x: icd_code in x)]['hadm_id'].values

    TP = np.intersect1d(true_icd_codes, predicted_icd_codes)
    FP = np.setdiff1d(predicted_icd_codes, true_icd_codes)
    FN = np.setdiff1d(true_icd_codes, predicted_icd_codes)
    TN = len(np.union1d(predicted_icd_codes, true_icd_codes)) -len(TP) -len(FP) -len(FN)


    filtered_counts[icd_code] = (len(TP), len(FP), len(FN), TN)

10 codes are done!
20 codes are done!
30 codes are done!
40 codes are done!
50 codes are done!


In [55]:
filtered_counts

{'N18.3': (0, 0, 1008, 0),
 'E87.2': (527, 224, 626, 0),
 'N40.0': (0, 0, 1030, 0),
 'J96.0': (286, 111, 786, 0),
 'G47.3': (848, 237, 1140, 0),
 'E78.5': (3422, 660, 3171, 0),
 'D72.8': (536, 2322, 419, 0),
 'Z95.5': (5, 5, 1167, 0),
 'I10.': (1028, 1709, 5492, 0),
 'I12.9': (28, 68, 1393, 0),
 'Z68.3': (0, 0, 1347, 0),
 'Y92.9': (0, 0, 1720, 0),
 'D64.9': (1278, 3053, 595, 0),
 'Y92.0': (0, 0, 1241, 0),
 'E11.4': (0, 0, 972, 0),
 'N18.9': (494, 925, 793, 0),
 'K21.9': (3549, 723, 962, 0),
 'I11.0': (1, 1, 1134, 0),
 'Z87.8': (14, 31, 5578, 0),
 'J44.9': (1224, 842, 249, 0),
 'Z86.7': (318, 773, 2474, 0),
 'I50.3': (0, 0, 1734, 0),
 'F41.9': (2097, 1833, 780, 0),
 'E11.2': (0, 0, 1501, 0),
 'G47.0': (778, 2347, 172, 0),
 'Z79.4': (0, 0, 2317, 0),
 'E87.1': (737, 357, 549, 0),
 'I25.1': (1881, 638, 1918, 0),
 'Y92.2': (0, 0, 2085, 0),
 'E66.9': (1159, 1411, 677, 0),
 'F17.2': (262, 199, 1712, 0),
 'Z96.6': (0, 0, 1004, 0),
 'F32.9': (2075, 1112, 1332, 0),
 'I50.2': (0, 0, 1294, 0),
 'I

In [56]:

# Calculate Precision and Recall for Each ICD-10 Code
filtered_precisions = {}
filtered_recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = filtered_counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    filtered_precisions[icd_code] = precision
    filtered_recalls[icd_code] = recall

In [57]:
filtered_precisions

{'N18.3': 0,
 'E87.2': 0.7017310252996005,
 'N40.0': 0,
 'J96.0': 0.7204030226700252,
 'G47.3': 0.7815668202764977,
 'E78.5': 0.8383145516903479,
 'D72.8': 0.18754373687893633,
 'Z95.5': 0.5,
 'I10.': 0.37559371574716843,
 'I12.9': 0.2916666666666667,
 'Z68.3': 0,
 'Y92.9': 0,
 'D64.9': 0.29508196721311475,
 'Y92.0': 0,
 'E11.4': 0,
 'N18.9': 0.34813248766737137,
 'K21.9': 0.8307584269662921,
 'I11.0': 0.5,
 'Z87.8': 0.3111111111111111,
 'J44.9': 0.5924491771539206,
 'Z86.7': 0.2914757103574702,
 'I50.3': 0,
 'F41.9': 0.533587786259542,
 'E11.2': 0,
 'G47.0': 0.24896,
 'Z79.4': 0,
 'E87.1': 0.6736745886654479,
 'I25.1': 0.7467248908296943,
 'Y92.2': 0,
 'E66.9': 0.4509727626459144,
 'F17.2': 0.5683297180043384,
 'Z96.6': 0,
 'F32.9': 0.6510825227486664,
 'I50.2': 0,
 'I48.9': 0.5237278444825615,
 'N39.0': 0.5434782608695652,
 'Z66.': 0,
 'I25.2': 0.5178571428571429,
 'G89.2': 0,
 'Z79.0': 0,
 'Z85.8': 0.5,
 'E11.9': 0.37874097007223945,
 'J45.9': 0.6586666666666666,
 'I48.0': 0.8276422

In [58]:
filtered_recalls

{'N18.3': 0.0,
 'E87.2': 0.4570685169124024,
 'N40.0': 0.0,
 'J96.0': 0.2667910447761194,
 'G47.3': 0.42655935613682094,
 'E78.5': 0.519035340512665,
 'D72.8': 0.5612565445026177,
 'Z95.5': 0.004266211604095563,
 'I10.': 0.15766871165644172,
 'I12.9': 0.019704433497536946,
 'Z68.3': 0.0,
 'Y92.9': 0.0,
 'D64.9': 0.6823278163374266,
 'Y92.0': 0.0,
 'E11.4': 0.0,
 'N18.9': 0.3838383838383838,
 'K21.9': 0.7867435158501441,
 'I11.0': 0.000881057268722467,
 'Z87.8': 0.002503576537911302,
 'J44.9': 0.8309572301425662,
 'Z86.7': 0.11389684813753581,
 'I50.3': 0.0,
 'F41.9': 0.7288842544316997,
 'E11.2': 0.0,
 'G47.0': 0.8189473684210526,
 'Z79.4': 0.0,
 'E87.1': 0.573094867807154,
 'I25.1': 0.4951302974466965,
 'Y92.2': 0.0,
 'E66.9': 0.6312636165577342,
 'F17.2': 0.13272543059777103,
 'Z96.6': 0.0,
 'F32.9': 0.6090402113296155,
 'I50.2': 0.0,
 'I48.9': 0.9294774226281075,
 'N39.0': 0.6810631229235881,
 'Z66.': 0.0,
 'I25.2': 0.04577742699289661,
 'G89.2': 0.0,
 'Z79.0': 0.0,
 'Z85.8': 0.0038

In [59]:
# Step 6: Calculate Macro Precision and Macro Recall
macro_precision = sum(filtered_precisions.values()) / len(filtered_precisions)
macro_recall = sum(filtered_recalls.values()) / len(filtered_recalls)


In [60]:
macro_precision

0.36444586070714435

In [61]:
macro_recall

0.29479248464857305

In [62]:
# Calculate F1 Score for Each ICD-10 Code
filtered_f1_scores = {}
for icd_code in unique_icd_codes:
    precision = filtered_precisions[icd_code]
    recall = filtered_recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    filtered_f1_scores[icd_code] = f1_score



In [63]:
# Step 8: Calculate Macro F1 Score
macro_f1_score = sum(filtered_f1_scores.values()) / len(filtered_f1_scores)

# Now you have macro precision, macro recall, and macro F1 score for your multi-label classification evaluation.

In [64]:
macro_f1_score

0.2693193129654034

In [65]:
# Calculate Precision and Recall for Each ICD-10 Code
filtered_precisions = {}
filtered_recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = filtered_counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    filtered_precisions[icd_code] = precision
    filtered_recalls[icd_code] = recall    
    
    
    
    
    
# Filter out zero precision values and calculate average precision
average_precision = np.mean([precision for precision in filtered_precisions.values() if precision > 0])

In [66]:
average_precision

0.5359497951575651

In [67]:
average_recall = np.mean([recall for recall in filtered_recalls.values() if recall > 0])

In [68]:
average_recall

0.4335183597773133

In [69]:
# Calculate F1 Score for Each ICD-10 Code
filtered_f1_scores = {}
for icd_code in unique_icd_codes:
    precision = filtered_precisions[icd_code]
    recall = filtered_recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    filtered_f1_scores[icd_code] = f1_score


In [70]:
average_f1 = np.mean([f1_score for f1_score in filtered_f1_scores.values() if f1_score > 0])

In [71]:
average_f1

0.39605781318441674